### check or install cuda version, e.g.,conda install cudatoolkit=10.1
```
##pip install cellpose
pip uninstall mxnet-mkl
pip uninstall mxnet
pip install mxnet-cu101 
conda install -c anaconda ipython
```
### imjoy
```
pip install imjoy
pip install -U imjoy-jupyter-extension

```

In [4]:
from imjoy import api
import numpy as np
import asyncio
# import imageio

class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        viewer = await api.createWindow(src="https://kaibu.org/#/app")

        # create a random image
        image = np.random.randint(0, 255, [500, 500], dtype='uint8')
        
        # or you can try if you also did `pip install imageio` and `import imageio`
        # image = imageio.imread("https://images.proteinatlas.org/19661/221_G2_1_red_green.jpg")

        # view image
        await viewer.view_image(image, type="itk-vtk", name="random pixels")

api.export(ImJoyPlugin())

<IPython.core.display.Javascript object>